In [34]:
import nibabel as nib
import numpy as np
import plotly.express as px
import plotting_utils as pu
import glob
import pandas as pd
import os
import torch
import importlib
import sys
sys.path.append("/projects/master-theses/henschell")
from FastInfantSurfer.data_util import data_utils as du

In [36]:
importlib.reload(du)

<module 'FastInfantSurfer.data_util.data_utils' from '/projects/master-theses/henschell/FastInfantSurfer/data_util/data_utils.py'>

In [2]:
# Load orig image, ground truth image
data = "/projects/datasets/dHCP/Data"
sbj="sub-CC00189XX17_ses-62301"
gts=sbj + "_desc-drawem88_dseg"

orig = nib.load(os.path.join(data, sbj, "T2w.nii.gz"))
orig_e = nib.load(os.path.join(data, sbj, "T2w_08_dhcp.nii.gz"))
orig_min = nib.load(os.path.join(data, sbj, "T2w_min.nii.gz"))

gt = nib.load(os.path.join(data, sbj, gts + ".nii.gz"))
gt_e = nib.load(os.path.join(data, sbj, gts + "_08.nii.gz"))
gt_min = nib.load(os.path.join(data, sbj, gts + "_min.nii.gz"))

In [8]:
def rescale_probs(probs, rescale_shape):
    """
    Rescale probabilities:
      - (move class channel to front, )
      - interpolate to shape defined by out_scale_factor
      - (move class channel to back)
    :params torch.tensor probs: predicted probabilities for image (CxHxWxD)
    :params tuple rescale_shape: Output shape for rescaled probability map (HxWxD)
    :return torch.tensor: rescaled probabilities for image (CxHxWxD)
    """
    if isinstance(probs, np.ndarray):
        # to torch tensor
        probs = torch.from_numpy(probs)
    probs = probs.unsqueeze(0)
    probs = torch.nn.functional.interpolate(probs, size=rescale_shape, 
                                            mode="trilinear", align_corners=False)
    return torch.squeeze(probs, 0)

In [9]:
def to_one_hot(seg, all_seg_labels=None):
    if all_seg_labels is None:
        all_seg_labels = np.unique(seg)
    result = np.zeros((len(all_seg_labels), *seg.shape), dtype=seg.dtype)
    for i, l in enumerate(all_seg_labels):
        result[i][seg == l] = 1
    return result

In [10]:
gt_array = np.asanyarray(gt.dataobj, dtype=np.float32)
gt_e_array = np.asanyarray(gt_e.dataobj, dtype=np.int32)
gt_min_array = np.asanyarray(gt_min.dataobj, dtype=np.int32)
gt_array_oh = to_one_hot(gt_array)
gt_array_re_min = rescale_probs(gt_array_oh, (512, 512, 512))

In [7]:
_, pred = torch.max(gt_array_re_min, 0)
pred.shape

torch.Size([512, 512, 512])

In [12]:
gt_e.affine

array([[ -0.80000001,   0.        ,   0.        ,  99.32225037],
       [  0.        ,   0.        ,   0.80000001, -86.29715729],
       [  0.        ,  -0.80000001,   0.        , 124.35568237],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [20]:
# Problem: output of mri_convert is not 512, but 290 --> how does this need to be cropped
# to accurately present the image? Need to get origin stuff from affine and calculate it
gt_min.affine

array([[ -0.5       ,   0.        ,   0.        ,  69.42224121],
       [  0.        ,   0.        ,   0.5       , -56.39715576],
       [  0.        ,  -0.5       ,   0.        ,  94.45568848],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [61]:
hallo = gt.affine
hallo

array([[  -1.        ,    0.        ,    0.        ,  124.92224121],
       [   0.        ,    0.        ,    1.        , -111.89715576],
       [   0.        ,   -1.        ,    0.        ,  149.95568848],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [67]:
# vox2ras
hallo[:3, :3] = hallo[:3, :3] * 0.5
hallo

array([[  -0.5       ,    0.        ,    0.        ,  124.92224121],
       [   0.        ,    0.        ,    0.5       , -111.89715576],
       [   0.        ,   -0.5       ,    0.        ,  149.95568848],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [79]:
# zooms = The size of voxels of the output affine
# new_shape = The extent of the (N-1) dimensions of the space described by the new affine. If None, use shape.
rescaled_affine = nib.affines.rescale_affine(gt.affine, gt.shape, 0.5, new_shape=(512, 512, 512))
rescaled_affine

array([[  -0.5       ,    0.        ,    0.        ,  125.42224121],
       [   0.        ,    0.        ,    0.5       , -112.39715576],
       [   0.        ,   -0.5       ,    0.        ,  150.45568848],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [23]:
def rescale_affine(affine, shape, zooms, new_shape=None):
    """ Return a new affine matrix with updated voxel sizes (zooms)
    This function preserves the rotations and shears of the original
    affine, as well as the RAS location of the central voxel of the
    image.
    Parameters
    ----------
    affine : (N, N) array-like
        NxN transform matrix in homogeneous coordinates representing an affine
        transformation from an (N-1)-dimensional space to an (N-1)-dimensional
        space. An example is a 4x4 transform representing rotations and
        translations in 3 dimensions.
    shape : (N-1,) array-like
        The extent of the (N-1) dimensions of the original space
    zooms : (N-1,) array-like
        The size of voxels of the output affine
    new_shape : (N-1,) array-like, optional
        The extent of the (N-1) dimensions of the space described by the
        new affine. If ``None``, use ``shape``.
    Returns
    -------
    affine : (N, N) array
        A new affine transform with the specified voxel sizes
    """
    shape = np.array(shape, copy=False)
    new_shape = np.array(new_shape if new_shape is not None else shape)

    s = nib.affines.voxel_sizes(affine)
    rzs_out = affine[:3, :3] * zooms / s

    # Using xyz = A @ ijk, determine translation
    centroid = nib.affines.apply_affine(affine, (shape - 1) // 2)
    t_out = centroid - rzs_out @ ((new_shape - 1) // 2)
    return nib.affines.from_matvec(rzs_out, t_out)

In [24]:
rescaled_affine = rescale_affine(gt.affine, gt.shape, 0.8, new_shape=(256, 256, 256))
rescaled_affine

array([[ -0.8       ,   0.        ,   0.        ,  99.52224121],
       [  0.        ,   0.        ,   0.8       , -86.49715576],
       [  0.        ,  -0.8       ,   0.        , 124.55568848],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [80]:
test_header = gt.header
test_header.set_data_shape((512, 512, 512))
test_header.set_zooms(zooms=(0.5, 0.5, 0.5))
test_header.set_sform(rescaled_affine)
print(test_header.get_best_affine())

[[  -0.5           0.            0.          125.42224121]
 [   0.            0.            0.5        -112.39715576]
 [   0.           -0.5           0.          150.45568848]
 [   0.            0.            0.            1.        ]]


In [81]:
du.save_image_as_nifti(test_header, rescaled_affine, pred.numpy(), os.path.join(data, sbj, "test_supersample.mgz"))

In [52]:
print(gt_min.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 290 290 290   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.     0.5    0.5    0.5    0.001  1.     1.     1.   ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'FreeSurfer Jul 20 2021'
aux_file        : b''
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 0.70710677
quatern_d       : -0.70710677
qoffset_x       : 69.42224
qoffset_y       : -56.3971

In [74]:
# super 
# [0, 0, 0] --> [-112, -112, -112]
# [512, 0, 0] --> [400, -112, -112]
# [0, 512, 0] --> [-112, 400, -112]
# [0, 0, 512] --> [-112, -112, 400]
# [512, 512, 512] --> [400, 400, 400]

# [0, 0, 0] --> [112, 112, 112]
# [290, 0, 0] --> [402, 112, 112]
# [0, 290, 0] --> [112, 402, 112]
# [0, 0, 290] --> [112, 112, 402]
# [290, 290, 290] --> [402, 402, 402]
print(f"{512-290}, {(512-290)/2}, {512-(512-290)/2}")
402-111

222, 111.0, 401.0


291

In [82]:
# cut image 512 --> 
pred_cut = pred[112:403, 112:403, 112:403]

du.save_image_as_nifti(gt_min.header, gt_min.affine, pred_cut.numpy(), os.path.join(data, sbj, "test_supersample_rlpy.mgz"))

In [119]:
def crop_image_to_minsize(orig, gt_min, img, zoom, save_as=None):
    ras_zero = np.asarray([0, 0, 0, 1])
    ras_zero_vox_min = np.linalg.inv(gt_min.affine) @ ras_zero
    
    rescaled_affine = nib.affines.rescale_affine(orig.affine, orig.shape, zoom, 
                                                 new_shape=np.asarray(orig.shape)/zoom)
    ras_zero_vox_rescale = np.linalg.inv(rescaled_affine) @ ras_zero
    start = (ras_zero_vox_rescale[:-1] - ras_zero_vox_min[:-1]).astype(int)
    stop = start + gt_min.shape
    pred_cut = img[start[0]:stop[0], start[1]:stop[1], start[2]:stop[2]]
    if save_as is not None:
        du.save_image_as_nifti(gt_min.header, gt_min.affine, pred_cut, save_as)
    return pred_cut
    

In [130]:
gt_array_oh = to_one_hot(gt_array)
target_zoom = 0.8
gt_array_re_min = rescale_probs(gt_array_oh, (320, 320, 320))
_, pred = torch.max(gt_array_re_min, 0)
out = crop_image_to_minsize(gt, gt_e, pred, target_zoom, os.path.join(data, sbj, "test_supersample_rlpy_08.mgz"))

In [131]:
out.shape

torch.Size([256, 256, 256])

In [123]:
np.asarray(gt.shape) / 0.8

array([320., 320., 320.])

In [104]:
pred.shape

torch.Size([512, 512, 512])

In [114]:
pred[b[0]:stop[0], b[1]:stop[1], b[2]:stop[2]].shape

torch.Size([290, 290, 290])

In [37]:
def crop_image_from_affine(large_img, small_img, save_as=None, dtype=None):
    if isinstance(large_img, str):
        large_img = nib.load(large_img)
    if isinstance(small_img, str):
        small_img = nib.load(small_img)
    ras_zero = np.asarray([0, 0, 0, 1])
    ras_zero_vox_min = np.linalg.inv(small_img.affine) @ ras_zero
    ras_zero_vox_max = np.linalg.inv(large_img.affine) @ ras_zero
    start = (ras_zero_vox_max[:-1] - ras_zero_vox_min[:-1]).astype(int)
    stop = start + small_img.shape
    large_cut = np.asanyarray(large_img.dataobj)[start[0]:stop[0], start[1]:stop[1], start[2]:stop[2]]
    if save_as is not None:
        du.save_image_as_nifti(small_img.header, small_img.affine, large_cut, save_as, dtype=dtype)
    return large_cut

In [38]:
testbase = "/projects/datasets/dHCP/Data/sub-CC00362XX08_ses-114500"
sampled = os.path.join(testbase, "T2w_1to5.nii.gz")
smallorig = os.path.join(testbase, "T2w_min.nii.gz")
smallsampled = os.path.join(testbase, "T2w_1to5_cropped.nii.gz")

sampled_small = crop_image_from_affine(sampled, smallorig, smallsampled)

In [51]:
sbjs=pd.read_csv("/projects/master-theses/henschell/FastInfantSurfer/data/dataset_split_large_testing_t1t2.csv", 
                 names=["Subject"])
sbjs["ID"] = sbjs["Subject"].str.split("/").str[-1]

In [52]:
sbjs["ID"]

0      sub-CC00362XX08_ses-114500
1      sub-CC00754AN12_ses-223800
2       sub-CC00955XX15_ses-39730
3      sub-CC00284BN13_ses-111500
4       sub-CC00136BN13_ses-45000
                  ...            
102    sub-CC00418BN14_ses-125300
103     sub-CC00129AN14_ses-43500
104      sub-CC00907XX16_ses-4230
105     sub-CC00136AN13_ses-45100
106     sub-CC00067XX10_ses-20200
Name: ID, Length: 107, dtype: object

In [55]:
db = "/projects/datasets/dHCP/Data/"
pre="mri/aseg.TestingSetRerun_FastSurferVINN_T2_"
suf="_T2_LatentAug_full.mgz"
for sbj in sbjs["ID"]:
    halfmin = os.path.join(db, sbj, "T2w_min.nii.gz")
    for half_cuts in ["T2w_1to5.nii.gz", "T2w_8to5.nii.gz", f"{pre}10NN05{suf}", f"{pre}08NN05{suf}"]:
        sampled = os.path.join(db, sbj, half_cuts)
        print(f"Cropping {sampled}")
        try:
            _ = crop_image_from_affine(sampled, halfmin, sampled)
        except FileNotFoundError as e:
            print(f"Processing failed on {sbj}: {e}")
    
    halfmin = os.path.join(db, sbj, "T2w_08_dhcp.nii.gz")
    for eight_cut in ["T2w_1to8.nii.gz", f"{pre}10NN08{suf}"]:
        sampled = os.path.join(db, sbj, eight_cut)
        print(f"Cropping {sampled}")
        try:
            _ = crop_image_from_affine(sampled, halfmin, sampled)
        except FileNotFoundError as e:
            print(f"Processing failed on {sbj}: {e}")

Cropping /projects/datasets/dHCP/Data/sub-CC00362XX08_ses-114500/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00362XX08_ses-114500/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00362XX08_ses-114500/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00362XX08_ses-114500/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00362XX08_ses-114500/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00362XX08_ses-114500/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00754AN12_ses-223800/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00754AN12_ses-223800/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00754AN12_ses-223800/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC007

Cropping /projects/datasets/dHCP/Data/sub-CC00254XX07_ses-84300/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00254XX07_ses-84300/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00181XX09_ses-60000/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00181XX09_ses-60000/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00181XX09_ses-60000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00181XX09_ses-60000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00181XX09_ses-60000/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00181XX09_ses-60000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00998AN26_ses-34030/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00998AN26_se

Cropping /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-27411/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-27411/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-27411/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-27411/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-27411/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00829XX21_ses-27411/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00343XX14_ses-108500/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00343XX14_ses-108500/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00343XX14_ses-108500/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00343XX14

Cropping /projects/datasets/dHCP/Data/sub-CC01044XX14_ses-72531/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC01044XX14_ses-72531/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC01044XX14_ses-72531/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00441XX13_ses-132902/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_ful

Cropping /projects/datasets/dHCP/Data/sub-CC00203XX05_ses-69500/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00203XX05_ses-69500/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00203XX05_ses-69500/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00422XX10_ses-127800/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00422XX10_ses-127800/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00422XX10_ses-127800/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00422XX10_ses-127800/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00422XX10_ses-127800/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00422XX10_ses-127800/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_ful

Cropping /projects/datasets/dHCP/Data/sub-CC00135AN12_ses-54400/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00135AN12_ses-54400/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00135AN12_ses-54400/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00135AN12_ses-54400/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00135AN12_ses-54400/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC01222XX10_ses-149231/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC01222XX10_ses-149231/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC01222XX10_ses-149231/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC01222XX10_ses-149231/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.

Cropping /projects/datasets/dHCP/Data/sub-CC00206XX08_ses-70100/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00206XX08_ses-70100/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00206XX08_ses-70100/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00124XX09_ses-42302/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00124XX09_ses-42302/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00124XX09_ses-42302/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00124XX09_ses-42302/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00124XX09_ses-42302/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00124XX09_ses-42302/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz


Cropping /projects/datasets/dHCP/Data/sub-CC00770XX12_ses-238000/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00770XX12_ses-238000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00770XX12_ses-238000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00770XX12_ses-238000/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00770XX12_ses-238000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00513XX10_ses-150000/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00513XX10_ses-150000/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00513XX10_ses-150000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00513XX10_ses-150000/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_

Cropping /projects/datasets/dHCP/Data/sub-CC01212XX08_ses-146030/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00466AN13_ses-138400/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00466AN13_ses-138400/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00466AN13_ses-138400/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00466AN13_ses-138400/mri/aseg.TestingSetRerun_FastSurferVINN_T2_08NN05_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00466AN13_ses-138400/T2w_1to8.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00466AN13_ses-138400/mri/aseg.TestingSetRerun_FastSurferVINN_T2_10NN08_T2_LatentAug_full.mgz
Cropping /projects/datasets/dHCP/Data/sub-CC00649XX23_ses-191201/T2w_1to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC00649XX23_ses-191201/T2w_8to5.nii.gz
Cropping /projects/datasets/dHCP/Data/sub-CC006